In [40]:
%%bash --out windowID --err err
# find simulator window id
xdotool search --name 'self_driving_car_nanodegree_program'

In [41]:
if (windowID == ''):
    print("ERROR: no window id found")
else:
    print("found following windowID: " + str(windowID))
    windowID = str(windowID)

found following windowID: 41943046



In [46]:
import random
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import time
import os
%matplotlib inline  
sleepTime = 0.5


def alarm():
    duration = 0.5  # second
    freq = 240  # Hz
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

def startSimulation():
    # start current simulation
    # uncomment to activate window
    #subprocess.call(["xdotool", "windowactivate", windowID])
    #time.sleep(sleepTime)
    subprocess.call(["xdotool", "mousemove", "596", "359"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "mousemove", "369", "445"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    

def exitSimulation():
    # exit current simulation
    # uncomment to activate window
    #subprocess.call(["xdotool", "xdotool", "windowactivate", windowID])
    #time.sleep(sleepTime)
    subprocess.call(["xdotool", "mousemove", "389", "41"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"]) 
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "key", "Escape"])
    time.sleep(sleepTime)

# run the cpp PID controller
def startPIDcontroller(params, N):
    process = subprocess.Popen(["./build/pid", str(params[0]), str(params[1]), str(params[2]), str(N)], stdout=subprocess.PIPE)
    out, err = process.communicate()
    return out

# NOTE: We use params instead of tau_p, tau_d, tau_i
def run(params, n=1400):
    startSimulation()
    err = startPIDcontroller(params, n)
    print(err)
    writeRunToFile("./twiddle.out", params, float(err)/n)
    exitSimulation()
    return float(err)/n

# write each simulator run to a file
def writeRunToFile(fn, p, best_err):
    f = open(fn, "a")
    f.write("new run: " + str(p) + ": " + str(best_err) + "\n")
    f.close()
    
# write parameter to a file
def writeParamsToFile(fn, dp, weights, p, best_err):
    f = open(fn, "a")
    f.write("-->changed dp: "+ str(dp) + " (sum: " + str(sum([float(a)*b for a,b in zip(dp,weights)])) + ")" + 
            ", p: " + str(p) + ", best_err: " + str(best_err) + "\n")
    f.close()

# Make this tolerance bigger if you are timing out!
def twiddle(start_p = [0.1, 0.001, 1.0], 
            start_dp = [1,1,1],
            weights=[10,1000,1], 
            tol=0.01): 
    p = start_p
    dp = start_dp
    dp = [float(a)/b for a,b in zip(dp,weights)]
    best_err = run(p)
    writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
    
    # twiddle loop
    while sum([float(a)*b for a,b in zip(dp,weights)]) > tol:
        for i in range(len(dp)):
            p[i] += dp[i]
            err = run(p)
            if err < best_err:
                best_err = err
                dp[i] *= 1.1
                writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
                
            else:
                p[i] -= 2*dp[i]
                err = run(p)
                if err < best_err:
                    best_err = err
                    dp[i] *= 1.1
                    writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
                else:
                    p[i] += dp[i];
                    dp[i]*=0.9
                    writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
    
    return p, best_err


params, err = twiddle()
print("Final twiddle error = {}".format(err))


b'680.909'
b'359.609'
b'360.233'
b'436.737'
b'323.812'
b'318.398'
b'290.13'
b'245.497'
b'277.688'
b'355.305'
b'267.349'
b'248.856'
b'236.839'
b'193.984'
b'186.329'
b'193.702'
b'308.593'
b'174.183'
b'165.395'
b'154.226'
b'152.089'
b'129.865'
b'140.397'
b'170.873'
b'135.265'
b'151.091'
b'133.769'
b'145.84'
b'136.459'
b'161.154'
b'151.229'
b'147.226'
b'123.082'
b'136.374'
b'160.533'
b'140.528'
b'138.736'
b'134.059'
b'145.906'
b'134.559'
b'153.502'
b'134.42'
b'144.198'
b'139.176'
b'135.408'
b'133.234'
b'146.582'
b'131.813'
b'142.473'
b'145.507'
b'135.369'
b'121.582'
b'122.346'
b'130.471'
b'130.747'
b'121.998'
b'133.759'
b'130.747'
b'129.536'
b'140.063'
b'131.806'
b'142.608'
b'126.01'
b'141.765'
b'123.152'
b'132.172'
b'139.998'
b'177.473'
b'133.13'
b'169.159'
b'128.188'
b'136.575'
b'127.405'
b'170.07'
b'125.311'
b'125.934'
b'125.73'
b'136.098'
b'122.539'
b'136.602'
b'182.6'
b'123.45'
b'137.326'
b'139.578'
b'127.036'
b'127.632'
b'127.068'
b'122.019'
b'134.401'
b'145.73'
b'121.9'
b'145.633'
b

KeyboardInterrupt: 

In [29]:
params

[0.18036159739366794, 9.999999999998899e-05, 4.286364811121022]